In [6]:
import pandas as pd
import numpy as np
class Rule:
    def __init__(self, class_label):
        self.conditions = []  # list of conditions
        self.class_label = class_label  # rule class
        self.accuracy = 0
        self.coverage = 0

    def addCondition(self, condition):
        self.conditions.append(condition)

    def setParams(self, accuracy, coverage):
        self.accuracy = accuracy
        self.coverage = coverage
    
    # Human-readable printing of this Rule
    def __repr__(self):
        return "If {} then {}. Coverage:{}, accuracy: {}".format(self.conditions, self.class_label,
                                                                 self.coverage, self.accuracy)
class Condition:
    def __init__(self, attribute, value, true_false = None):
        self.attribute = attribute
        self.value = value
        self.true_false = true_false

    def __repr__(self):
        if self.true_false is None:
            return "{}={}".format(self.attribute, self.value)
        else:
            return "{}>={}:{}".format(self.attribute, self.value, self.true_false)
min_coverage = 1
min_accuracy = 0.95
class_label = 2
data_file = "contact_lenses.csv"
data = pd.read_csv(data_file, index_col=['id'])
covered_subset = data.copy()
data2 = data.copy()
current_rule = Rule(class_label)
done = False
# filter out data with right labels
classheader = data2.columns[-1]
print(classheader)
current_data = data2[data2[classheader] == class_label]
current_accuracy = 0
current_coverage = 0
current_condition = Condition('tear production rate',2, True)

while not done:
    for i in range(len(current_data.columns) - 1):
        # make sure I am not repeatedly testing on the same criteria
        flag = False
        current_attribute = current_data.columns[i]
        print(current_attribute)
        for condition in current_rule.conditions:
            if current_attribute == condition.attribute:
                flag = True
        if flag == True:
            continue
        
        #test every possible value for the current attribute
        #check if it is numerical, get unique values, for each value, there is two rules: greater or equal to, or less than
        possible_values = current_data[current_attribute].unique().tolist()
        for value in possible_values:
            print(value)
            Int = False
            GreaterThan = False
            if isinstance(value, int):
                column = current_data[current_attribute]
                correct1 = column[column >= value].count()
                #problem: how do I fix the age/numeric data? Or is it possible for now to categorize?
                column = data2[current_attribute]
                coverage1 = column[column >= value].count()
                print(correct1, coverage1)
                if coverage1 == 0:
                        accuracy1 = 0
                else:
                    accuracy1 = correct1/coverage1
                column = current_data[current_attribute]
                correct = column[column <value].count()
                #problem: how do I fix the age/numeric data? Or is it possible for now to categorize?
                column = data2[current_attribute]
                coverage = column[column < value].count()
                if coverage == 0:
                    accuracy = 0
                else:
                    accuracy = correct/coverage
                GreaterThan = False
                if accuracy1 > accuracy:
                    GreaterThan = True
                    accuracy = accuracy1
                    coverage = coverage1
                elif accuracy1 == accuracy:
                    if coverage1 > coverage: 
                        GreaterThan = True
                        coverage = coverage1
                Int = True
                print(current_accuracy, accuracy)
                if accuracy > current_accuracy:
                    current_coverage = coverage
                    current_accuracy = accuracy
                    current_condition= Condition(current_attribute, value,GreaterThan) 
                    print(current_accuracy)
                    if GreaterThan: 
                        covered_subset = data2[data2[current_attribute] >= value] 
                    else:
                        covered_subset = data2[data2[current_attribute] < value] 
                # if accuracy is the same, compare coverage
                elif accuracy == current_accuracy and coverage > current_coverage:
                    current_coverage = coverage
                    current_accuracy = accuracy
                    current_condition= Condition(current_attribute, value,GreaterThan) 
                    if GreaterThan: 
                        covered_subset = data2[data2[current_attribute] >= value] 
                    else:
                        covered_subset = data2[data2[current_attribute] < value]        
                    
            else:
                correct = current_data[current_attribute].value_counts()[value]
                #problem: how do I fix the age/numeric data? Or is it possible for now to categorize?
                coverage = data2[current_attribute].value_counts()[value]
                accuracy = correct/coverage
                #choose the best option based on accuracy
                if accuracy > current_accuracy:
                    current_coverage = coverage
                    current_accuracy = accuracy
                    current_condition= Condition(current_attribute, value) 
                    covered_subset = data2[data2[current_attribute] == value] 
                # if accuracy is the same, compare coverage
                elif accuracy == current_accuracy and coverage > current_coverage:
                    current_coverage = coverage
                    current_accuracy = accuracy
                    current_condition= Condition(current_attribute, value) 
                    covered_subset = data2[data2[current_attribute] == value] 
                
                
    #if reached to the end accuracy = 1.0, add to rule and terminate
    if current_accuracy == 1.0 and current_coverage>min_coverage:
        done = True    
        current_rule.addCondition(current_condition)
        current_rule.accuracy = current_accuracy
        current_rule.coverage = current_coverage
    #if reached to the end that is acceptable, add to rule and terminate
    elif current_coverage <= min_coverage and current_accuracy > min_accuracy: 
        done = True
        current_rule.addCondition(current_condition)
        current_rule.accuracy = current_accuracy
        current_rule.coverage = current_coverage
    #if no rule possible, return none
    elif current_coverage <= min_coverage and current_accuracy <= min_accuracy:
        done = True
        print("failed")
    #default: not done yet, continue
    else: 
        current_rule.addCondition(current_condition)
        attribute = current_condition.attribute
        value = current_condition.value
        data2 = covered_subset    
        current_data = data2[data2[attribute] == value] 
        current_rule.accuracy = current_accuracy
        current_rule.coverage = current_coverage 
print(current_rule)

lenses type
age
1
5 24
0 0.20833333333333334
0.20833333333333334
2
3 16
0.20833333333333334 0.25
0.25
3
1 8
0.25 0.25
spectacles
1
5 24
0.25 0.20833333333333334
2
3 12
0.25 0.25
astigmatism
1
5 24
0.25 0.20833333333333334
tear production rate
2
5 12
0.25 0.4166666666666667
0.4166666666666667
age
1
12 12
0.4166666666666667 1.0
1.0
2
8 8
1.0 1.0
3
4 4
1.0 1.0
spectacles
1
12 12
1.0 1.0
2
6 6
1.0 1.0
astigmatism
1
12 12
1.0 1.0
2
6 6
1.0 1.0
tear production rate
If [tear production rate>=2:True, age>=1:True] then 2. Coverage:12, accuracy: 1.0


In [28]:
import pandas as pd
import numpy as np
data_file = "contact_lenses.csv"
data = pd.read_csv(data_file, index_col=['id'])
column = current_data['age']
possible_values = current_data['age'].unique().tolist()
print(possible_values)
print(column)
correct1 = column[column >= 1].count()
print(correct1)

[1, 2, 3]
id
2     1
6     1
10    2
14    2
22    3
Name: age, dtype: int64
5


In [ ]:
print(0/0)

ZeroDivisionError: division by zero

In [8]:
import pandas as pd
import numpy as np
class Rule:
    def __init__(self, class_label):
        self.conditions = []  # list of conditions
        self.class_label = class_label  # rule class
        self.accuracy = 0
        self.coverage = 0

    def addCondition(self, condition):
        self.conditions.append(condition)

    def setParams(self, accuracy, coverage):
        self.accuracy = accuracy
        self.coverage = coverage
    
    # Human-readable printing of this Rule
    def __repr__(self):
        return "If {} then {}. Coverage:{}, accuracy: {}".format(self.conditions, self.class_label,
                                                                 self.coverage, self.accuracy)
class Condition:
    def __init__(self, attribute, value, true_false = None):
        self.attribute = attribute
        self.value = value
        self.true_false = true_false

    def __repr__(self):
        if self.true_false is None:
            return "{}={}".format(self.attribute, self.value)
        else:
            return "{}>={}:{}".format(self.attribute, self.value, self.true_false)
min_coverage = 1
min_accuracy = 0.95
class_label = 2
data_file = "contact_lenses2.csv"
data = pd.read_csv(data_file, index_col=['id'])
covered_subset = data.copy()
data2 = data.copy()
current_rule = Rule(class_label)
done = False
# filter out data with right labels
classheader = data2.columns[-1]
print(classheader)
current_data = data2[data2[classheader] == class_label]
current_accuracy = 0
current_coverage = 0
current_condition = Condition('tear production rate',2, True)
while not done:
    for i in range(len(current_data.columns) - 1):
        # make sure I am not repeatedly testing on the same criteria
        flag = False
        current_attribute = current_data.columns[i]
        for condition in current_rule.conditions:
            if current_attribute == condition.attribute:
                flag = True
        if flag == True:
            continue
        
        #test every possible value for the current attribute
        #check if it is numerical, get unique values, for each value, there is two rules: greater or equal to, or less than
        possible_values = current_data[current_attribute].unique().tolist()
        for value in possible_values:
            correct = current_data[current_attribute].value_counts()[value]
            #problem: how do I fix the age/numeric data? Or is it possible for now to categorize?
            coverage = data2[current_attribute].value_counts()[value]
            accuracy = correct/coverage
            #choose the best option based on accuracy
            if accuracy > current_accuracy:
                current_coverage = coverage
                current_accuracy = accuracy
                current_condition= Condition(current_attribute, value) 
                covered_subset = data2[data2[current_attribute] == value] 
            # if accuracy is the same, compare coverage
            elif accuracy == current_accuracy and coverage > current_coverage:
                current_coverage = coverage
                current_accuracy = accuracy
                current_condition= Condition(current_attribute, value) 
                covered_subset = data2[data2[current_attribute] == value] 
    #if reached to the end accuracy = 1.0, add to rule and terminate
    if current_accuracy == 1.0 and current_coverage>min_coverage:
        done = True    
        current_rule.addCondition(current_condition)
        current_rule.accuracy = current_accuracy
        current_rule.coverage = current_coverage
    #if reached to the end that is acceptable, add to rule and terminate
    elif current_coverage <= min_coverage and current_accuracy > min_accuracy: 
        done = True
        current_rule.addCondition(current_condition)
        current_rule.accuracy = current_accuracy
        current_rule.coverage = current_coverage
    #if no rule possible, return none
    elif current_coverage <= min_coverage and current_accuracy <= min_accuracy:
        done = True
        print("failed")
    #default: not done yet, continue
    else: 
        current_rule.addCondition(current_condition)
        data2 = covered_subset    
        current_data = data2[data2[classheader] == class_label]  
print(current_rule)


lenses type
If [astigmatism=1, spectacles=2] then 2. Coverage:3, accuracy: 1.0
